**Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical, normalize

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

**Preprocessing the Dataset**

In [ ]:
datasetPath = '/content/drive/My Drive/My Project/'

IMG_SIZE = 50
CATEGORIES = ['no_mask', 'mask']
training_data = []


def create_training_data():
  for category in CATEGORIES:
    path = os.path.join(datasetPath, category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      training_data.append([new_array, category])

create_training_data()


In [ ]:
import random
random.shuffle(training_data)

In [ ]:
x=[]
y=[]


for features, labels in training_data:
    x.append(features)
    y.append(labels)

x = np.array(x, dtype="float32").reshape(-1,50,50,1)
y = np.array(y)

lbl= LabelBinarizer()
y=lbl.fit_transform(y)
y = to_categorical(y)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=42)


In [ ]:
#Create the Model

model = Sequential()
model.add(Conv2D(100, (3,3), activation='relu', padding='same', input_shape=(50,50,1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(100,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='sigmoid'))


#Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Fit the Model
model.fit(x_train,y_train,validation_data=(x_test, y_test) ,epochs=20,batch_size=10)


In [ ]:

!curl -o img.jpg https://zidbits.com/wp-content/uploads/2011/12/mask.jpg

In [ ]:
# Predict the Model

from google.colab.patches import cv2_imshow

img = cv2.imread("img.jpg", cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img,(50,50))
img = np.reshape(img,[-1,50,50,1])

classes = model.predict_classes(img)

print(classes)